# Implementation of a simple BRB system for predicting a non-linear 1D function

In this simple example, we will illustrate the basic usage of a BRB model by first constructing an initial rule base modeling a simple one dimensional non-linear function. We will asses the model by plotting the original function and the points predicted by the BRB model. Then, we will train the model using a randomly generated data set. Finally, we will re-asses the model by plotting the original function, the points predicted by BRB with the original rule base and with the trained rule base. We should expect a better performance by the trained model. 

This example is the same example as what is presented in the article https://doi.org/10.1016/j.eswa.2011.04.077

The function we will try to model is
$$f(x) = x\sin{x^2},\quad0\le x \le 3.$$

We begin by importing relevant modules and by defining the function and it's domain.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from desdeo_brb.brb import BRB

# The function. The inputs to the BRB model are expected to be 2D arrays,
# with the i:th row representing the i:th sample and the j:th column
# the value of the j:th attribute in that sample. This is reflected
# in the argument given to our function.
def f(x):
    return x[0] * np.sin(x[0]**2)

# limits of the function
low = 0
high = 3

# evaluation points
n_eval = 100
xs_eval = np.linspace(0, 3, n_eval).reshape(-1, 1)
ys_eval = np.apply_along_axis(f, 1, xs_eval)

# plot the original function
plt.xlabel("x")
plt.ylabel("f")
plt.ylim(-3, 3)
plt.plot(xs_eval, ys_eval, label="f", ls="dotted")
plt.legend()

Next, we define a BRB model and build an initial rule base. This will require that we define:
    1. A set of precedents that will function as the set of referential values for the input attribute(s)
    2. A similar set of consequents for the output.
    3. A mapping (a set of rules) that defines output values for the precedents.

In [ ]:
# The precedents should be defined as a 2D array with the i:th row
# representing the referential values for the i:th attribute. We have just
# one input attribute (x), therefore, we have just one row of precedents.
precedents = np.array([[0, 0.5, 1, 1.5, 2, 2.5, 3]])

# The consequents are also defined as a 2D array. However, the BRB model currently
# implemented support only the output of a sigle belief distribution regardless of
# the dimensions of input the attributes. This is expected behavior.
consequents = np.array([[-2.5, -1, 1, 2, 3]])

# Construct the initial BRB. Note that the 3rd argument is the original function,
# which in this case, works also as the mapping from consequents to expected output.
brb = BRB(precedents, consequents, f=f)
print("BRB model before training:")
print(brb)

From the above, we can see that an initial BRB model has been constructed successfully with an initial rule base. We can try to predict points with it. However, before we do so, we should briefly inspect the output of a prediction.

In [ ]:
res = brb.predict(np.array([[0.27]]))
print(res)

This is a belief distribution consisting of a list of tuples. The first element in each tuple if the belief degree that the output maps to a consequent, which is represented as the second element in the tuple. The object returned by `brb.predict` is a named tuple with the fields `precedents`, `consequents`, `predecents_belief_degrees` and `consequent_belief_degreed`. The results of a BRB prediction is meant to convey enough information so that we can employ easily (later on) evidential reasoning on the output. For example, if we want to construct an explanation. 

Anyways, keeping in mind the fields defined in the object returned by a BRB prediction, we can plot the actual values predicted by the BRB using a simple utility function, which in this case, is just the identity function `lambda x: x` implemented implicitly.

In [ ]:
# predicted points with untrained BRB model
ys_untrained = np.array(
    [
        np.sum(res.consequents * res.consequent_belief_degrees)
        for res in map(brb.predict, xs_eval)
    ]
)

# plot the original function and the prediction given by the BRB
plt.plot(xs_eval, ys_eval, label="f", ls="dotted")
plt.plot(xs_eval, ys_untrained, label="Untrained output", ls="--")
plt.legend()

We can see that the preicted output is clearly not random and follows some sensical pattern imitating the original function. This is good. Next, we will train the model using randomly generated training points. Training is slow, so for the sake of this example, we will use manually specified intermediate points for x.

N.B. The object returned by `brb_flatten_parameters` is  a `Trainable`, which is just a named tuple with the fields
`flat_trainables`, `n_attributes`, `n_precedents`, `n_rules` and `n_consequents`. The `flat_trainables` contains all the parameters and the other fields are just used to construct the original parameters from the flattened array.

In [ ]:
# these points are between the precedetnts
n_train = 100
xs_train = np.random.uniform(low, high, (n_train, 1))
ys_train = np.apply_along_axis(f, 1, xs_train)

# The BRB model requires an initial guess for the parameters to be trained. We can use
# the model's own parameters defining the initial rule base as an initial guess.

brb.train(xs_train, ys_train, brb._flatten_parameters())
print("BRB after training:")
print(brb)

The training was (hopefully) successful. Finally, we will plot the trained model, the untrained model and the original function. We should expect better performance from the trained model.

In [ ]:
# predicted points with trained BRB model
ys_trained = np.array(
    [
        np.sum(res.consequents * res.consequent_belief_degrees)
        for res in map(brb.predict, xs_eval)
    ]
)

# plot the original function and the prediction given both by the trained
# and untrained BRB models
plt.plot(xs_eval, ys_eval, label="f", ls="dotted")
plt.plot(xs_eval, ys_untrained, label="Untrained output", ls="--")
plt.plot(xs_eval, ys_trained, label="Trained output")
plt.legend()

Due to the random nature of the training data, the results may vary. However, The trained model should still be noticeably better than the untrained one. This finishes our example.